In [2]:
import json

with open('two_way.json', 'r', encoding='utf8') as rp:
    a = json.load(rp)

In [22]:
a[0]['article text'][0]

{'entailment score': 0.7902937531471252,
 'comment': 'В том числе компания поставляет тепло и горячую воду предприятию, эксплуатирующему шахту "Интинская".',
 'text': 'Указанное понимание не соотносится и с буквальным смыслом абзаца второго пункта 1 статьи 134 Закона о банкротстве, закрепляющего единственный критерий внеочередного удовлетворения требований кредитора - направленность платежей на предотвращение катастроф либо гибели людей, вне зависимости от того, связано ли обязательство по внесению платежей с исполнением обязанностей по договору, заключенному в рамках обычной хозяйственной деятельности должника, или нет.'}

In [32]:
X = []
y = []
X = [a[num]['article text'][i]['text'] for num in range(len(a)) for i in range(len(a[num]['article text']))]
y = [a[num]['article text'][i]['comment'] for num in range(len(a)) for i in range(len(a[num]['article text']))]

In [125]:
import pandas as pd

X = pd.DataFrame(X)
y = pd.DataFrame(y)



In [126]:
X  # предложения из первых 100 статей. Они не разделены между собой.

,0
0,Указанное понимание не соотносится и с букваль...
1,3. Абзац второй пункта 1 статьи 134 Федерально...
2,Указанное понимание не соотносится и с букваль...
3,3. Абзац второй пункта 1 статьи 134 Федерально...
4,3. Абзац второй пункта 1 статьи 134 Федерально...
...,...
3287,"N 29, ст. 4339, 4350;"
3288,Реорганизация и ликвидация единого заказчика 1.
3289,Заключительные положения 1.
3290,Заключительные положения 1.


In [127]:
y

,0
0,В том числе компания поставляет тепло и горячу...
1,Шахта считается опасным объектом.
2,Поэтому после признания в 2019 году предприяти...
3,"Чтобы избежать возможного ЧП, эту теплоэнергию..."
4,Но это вызвало протест налоговой службы.
...,...
3287,Запрет и так не распространялся на большинство...
3288,Ранее Банк России решил не наказывать банки за...
3289,Контрольно-надзорные органы смогут прекращать ...
3290,"Сейчас если контролеры выявляют, что товар, на..."


In [128]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   shuffle=True)

In [130]:
len(X_train)

2633

In [131]:
len(X_test)

659

In [132]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [133]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

loading configuration file https://huggingface.co/DeepPavlov/rubert-base-cased-conversational/resolve/main/config.json from cache at C:\Users\Марк/.cache\huggingface\transformers\0a2d59d385ab4a44ba21fc77df723e894d643ab4844d4e72375aaf47008cbc26.e8f15c5aad2f4653e46ceeba0bb32c02a629d106a902c964bce60523d290ac8f
Model config BertConfig {
  "_name_or_path": "DeepPavlov/rubert-base-cased-conversational",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "fi

In [134]:
from transformers import TrainingArguments


training_args = TrainingArguments(output_dir='result_mag',
                                 num_train_epochs=5,
                                 learning_rate=6e-5,
                                 per_device_train_batch_size=64,
                                 per_device_eval_batch_size=64,
                                 weight_decay=0.01,
                                 logging_steps=100
                                 )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [137]:
from transformers import Trainer

trainer =  Trainer(model=model,
                   args=training_args,
                   train_dataset=[X_train, y_train],
                   eval_dataset=[X_test, y_test],
                    tokenizer=tokenizer)

In [138]:
trainer.train()

C:\conda\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 5


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
from sklearn.metrics import f1_score


f1 = f1_score()